In [8]:

# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random
import torchvision.models as models

from lihongyi.hw2.HW2 import best_acc




### 处理数据label

In [9]:
# 处理train_csv的label
train_df = pd.read_csv('./data/train.csv')
unique_labels = train_df['label'].unique()

# 创建映射字典
label_to_number = {label: idx + 1 for idx, label in enumerate(unique_labels)}

# 打印映射结果
print("Label to number mapping:", label_to_number)

# 应用映射字典
train_df['label_encoded'] = train_df['label'].map(label_to_number)

# 加1是为了从1开始编号
train_array = train_df.to_numpy()
print(train_array.shape)


Label to number mapping: {'maclura_pomifera': 1, 'ulmus_rubra': 2, 'broussonettia_papyrifera': 3, 'prunus_virginiana': 4, 'acer_rubrum': 5, 'cryptomeria_japonica': 6, 'staphylea_trifolia': 7, 'asimina_triloba': 8, 'diospyros_virginiana': 9, 'tilia_cordata': 10, 'ulmus_pumila': 11, 'quercus_muehlenbergii': 12, 'juglans_cinerea': 13, 'cercis_canadensis': 14, 'ptelea_trifoliata': 15, 'acer_palmatum': 16, 'catalpa_speciosa': 17, 'abies_concolor': 18, 'eucommia_ulmoides': 19, 'quercus_montana': 20, 'koelreuteria_paniculata': 21, 'liriodendron_tulipifera': 22, 'styrax_japonica': 23, 'malus_pumila': 24, 'prunus_sargentii': 25, 'cornus_mas': 26, 'magnolia_virginiana': 27, 'ostrya_virginiana': 28, 'magnolia_acuminata': 29, 'ilex_opaca': 30, 'acer_negundo': 31, 'fraxinus_nigra': 32, 'pyrus_calleryana': 33, 'picea_abies': 34, 'chionanthus_virginicus': 35, 'carpinus_caroliniana': 36, 'zelkova_serrata': 37, 'aesculus_pavi': 38, 'taxodium_distichum': 39, 'carya_tomentosa': 40, 'picea_pungens': 41, '

### DataSet&Transform

In [10]:

train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


# 两种形式，一种传file，一种直接传Tensor
# 验证集使用K折验证

# 存图片类型，getItem时，才转为Tensor
class LeavesFileDataset(Dataset):
    def __init__(self, path, train_array, tfm=test_tfm):
        super(LeavesFileDataset).__init__()
        self.path = path
        self.train_array = train_array
        self.transform = tfm

    def __len__(self):
        return len(self.train_array)

    def __getitem__(self, idx):
        data = self.train_array[idx]

        im = Image.open(self.path + data[0])
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = data[2]
        except:
            label = -1  # test has no label
        return im, label


# 存张量类型，可能比较消耗内存
class LeavesTensorDataset(Dataset):
    def __init__(self, X, y=None):
        super(LeavesTensorDataset).__init__()
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]





### 超参数


In [11]:
patience = 6
n_epochs = 2
batch_size = 64
train_ratio = 0.95  # the ratio of data used for training, the rest will be used for validation
# training parameters
seed = 0  # random seed

_exp_name = 'Leaves'

learning_rate = 0.0001  # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved
device = "cuda" if torch.cuda.is_available() else "cpu"
# model parameters



In [26]:
stale = 0
def train(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device, best_acc):

    # These are used to record information in training.


    for epoch in range(n_epochs):
        model.train()
        print('best_acc:' + str(best_acc))
       
        train_loss = []
        train_accs = []
        for batch in tqdm(train_loader):
            imgs, labels = batch
            # 

            labels = labels.to(device)
            # 梯度设置下 
            optimizer.zero_grad()

            logits = model(imgs.to(device))
            loss = criterion(logits, labels)

            # 反向传播
            loss.backward()

            # Clip the gradient norms for stable training. 
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

            # Update the parameters with computed gradients.
            optimizer.step()

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            train_loss.append(loss.item())
            train_accs.append(acc)

        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        # Print the information.
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()

        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []

        # Iterate the validation set by batches.
        for batch in tqdm(valid_loader):
            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()

            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model(imgs.to(device))

            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            valid_loss.append(loss.item())
            valid_accs.append(acc)
        #break

        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # update logs
        if valid_acc > best_acc:
            with open(f"./{_exp_name}_log.txt", "a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt", "a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"{_exp_name}_best.ckpt")  # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break







In [27]:

_dataset_dir = ".\\data\\"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
print(len(train_array))
train_set = LeavesFileDataset(os.path.join(_dataset_dir), train_array,tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)







18353


### K折交叉验证

In [28]:
import torchvision.models as models

def k_fold_split(dataset, k_folds=5):
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    fold_size = len(dataset) // k_folds
    folds = [indices[i * fold_size:(i + 1) * fold_size] for i in range(k_folds)]
    
    return folds

k_folds = 3
print('train length:'+ str(len(train_set)))
folds = k_fold_split(train_set, k_folds)
print('folds:'+ str(len(folds)))
for fold in range(k_folds):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # 定义模型、损失函数、优化器等
    model = models.resnet18()
    # model = ...  # 初始化模型
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5) 


    # 根据索引创建子集
    train_ids = [idx for f in folds[:fold] + folds[fold+1:] for idx in f]
    valid_ids = folds[fold]

    train_subsampler = Subset(train_set, train_ids)
    print('train length:' + str( len(train_subsampler)))
    valid_subsampler = Subset(train_set, valid_ids)
    print('valid length:' + str( len(valid_subsampler)))

    # 创建 DataLoader
    train_loader = DataLoader(train_subsampler, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_subsampler, batch_size=batch_size, shuffle=False)

    # 调用你的训练函数
    best_acc = 0
    train(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device,best_acc)

train length:18353
folds:3
FOLD 0
--------------------------------
train length:12234
valid length:6117


  0%|          | 0/192 [00:00<?, ?it/s]

[ Train | 001/002 ] loss = 4.73606, acc = 0.07983


  0%|          | 0/96 [00:00<?, ?it/s]

[ Valid | 001/002 ] loss = 3.63940, acc = 0.16522
[ Valid | 001/002 ] loss = 3.63940, acc = 0.16522 -> best
Best model found at epoch 0, saving model


...

[ Train | 001/002 ] loss = 4.71625, acc = 0.08869


  0%|          | 0/96 [00:00<?, ?it/s]

[ Valid | 001/002 ] loss = 3.79091, acc = 0.14859
[ Valid | 001/002 ] loss = 3.79091, acc = 0.14859 -> best
Best model found at epoch 0, saving model


  0%|          | 0/192 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
# 将生成的测试结果转为
def generate_pred(predictions):
    number_to_label = {v: k for k, v in label_to_number.items()}
    for i in range(predictions.shape[0]):
        predictions[i][1] = number_to_label[int(predictions[i][1])]

    with open('prediction.csv', 'w') as f:
        f.write('Id,Class\n')
        for i, y in enumerate(predictions):
            f.write('{},{}\n'.format(i, y))
 







[['images/0.jpg' 'maclura_pomi']
 ['images/1.jpg' 'broussonetti']
 ['images/2.jpg' 'broussonetti']
 ['images/3.jpg' 'prunus_virgi']
 ['images/4.jpg' 'acer_rubrum']]
